In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('qt5agg')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ncautil.ncalearn import *
from ncautil.seqgen import SeqGen
from ncautil.seqmultip import GRU_NLP
from ncautil.nlputil import NLPutil
import torch

/Users/zhengqihe/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
rnn=GRU_NLP(lsize,30,lsize)
rnnt=run_training(dataset, lsize, rnn, 2000, learning_rate=1e-2, batch=20, window=30)

cpu
Perlexity:  200 10.255487314365949
Perlexity:  400 8.867655811557652
Perlexity:  600 8.48778647081973
Perlexity:  800 8.261566572971978
Perlexity:  1000 7.216129414654686
Perlexity:  1200 8.184216008131473
Perlexity:  1400 7.402386472165774
Perlexity:  1600 7.734381473052133
Perlexity:  1800 7.57477874551553
Time used in training: 51.323198080062866


In [9]:
do_eval(dataset[:20000],lsize,rnnt)

Start Evaluation ...
torch.Size([46, 19999])
Evaluation Perplexity:  7.748471542261437


(tensor([[ -2.9008,  -3.3260,  -3.2582,  ...,  -3.2661,  -0.7397,
           -2.2360],
         [ -2.8398,  -2.5906,  -2.8366,  ...,  -2.4394,  -4.0588,
           -0.7384],
         [ -1.0844,  -1.5464,  -1.5849,  ...,  -4.2346,  -3.7072,
           -6.0036],
         ...,
         [-10.1075, -11.5644,  -8.1597,  ..., -12.3583,  -9.9199,
           -9.9725],
         [-11.4970, -13.1830, -11.0455,  ..., -12.0565, -12.9723,
          -11.0967],
         [ -8.8060, -10.0965, -10.5440,  ..., -13.2602, -13.3736,
          -11.4153]]),
 [tensor([[[-0.7249,  0.7301,  0.1627,  0.0328,  0.1530, -0.2518,  0.0959,
            -0.1076,  0.7081,  0.0300, -0.5247, -0.1242,  0.4541, -0.8426,
             0.4114, -0.3277,  0.3959,  0.3370, -0.1902, -0.3245,  0.1635,
             0.3604, -0.3924, -0.2690,  0.3878, -0.3039, -0.6459, -0.5072,
             0.2801, -0.5563]]]),
  tensor([[[-0.8256,  0.7631,  0.2192, -0.1495,  0.2152, -0.4217,  0.1253,
            -0.3989,  0.8180, -0.0006, -0.7842, -0.34

In [14]:
seq=[]
a0=np.zeros(lsize)
seq.append(a0)
x=np.zeros(lsize)
d=dataset[0]
x[d]=1
x=torch.from_numpy(x)
x=x.type(torch.FloatTensor)
hn=rnnt.initHidden(1)
for ii in range(len(dataset)-1):
    out,hn=rnnt.forward(x.view(1, 1, lsize), hn, logit_mode=True)
    logit=out.view(-1).data.numpy()
    seq.append(logit-np.mean(logit))
    d=dataset[ii+1]
    x=np.zeros(lsize)
    x[d]=1
    x=torch.from_numpy(x)
    x=x.type(torch.FloatTensor)
print(seq[0:100])

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([ 3.2519546 ,  3.1829462 ,  5.160517  ,  0.9184898 ,  1.0652982 ,
        2.7411728 ,  1.4714077 ,  4.3282237 ,  3.2071986 ,  2.1071403 ,
        3.0775058 ,  1.5598593 , -0.73324144,  3.0728548 ,  0.62680024,
       -0.74044   ,  2.6471124 , -0.92876494, -0.4631188 ,  0.86209106,
        1.9249706 ,  3.7621121 , -2.1724772 , -1.8940722 , -0.12593162,
       -0.7863921 ,  1.7374932 , -0.87739384, -1.4526323 ,  2.781851  ,
       -0.55622137, -2.9667087 , -1.5449346 , -1.3262717 , -2.0246577 ,
       -0.17524445,  0.5101783 , -3.9858627 , -2.571107  , -4.017651  ,
       -4.8538704 , -2.9998655 , -2.12257   , -3.9153452 , -4.258849  ,
       -2.5035563 ], dtype=float32), array([ 3.3474927 ,  3.9989395 ,  5.2501945 ,  0.7768042 ,  2.2831917 ,
        2.589377  ,  1.3310452 ,  5

In [23]:
resp=logit_space_transfer(seq)

In [24]:
labels,centers=cluster(resp,8)

In [25]:
vecres=[]
for ii in range(len(labels)):
    vecres.append(centers[labels[ii]])
vecres46=logit_space_atransfer(vecres)

In [29]:
len(labels)

100676

In [26]:
print(labels[0:100])

[7 3 3 5 3 1 2 5 7 0 6 2 4 6 6 1 3 1 5 3 3 0 2 4 3 3 5 6 3 6 1 5 3 3 5 3 1
 2 7 6 1 4 3 3 3 3 5 0 0 2 6 6 1 4 6 6 6 1 5 6 1 4 1 1 0 2 1 7 0 3 1 1 0 0
 6 6 2 4 2 1 4 6 1 4 1 2 2 4 1 2 4 2 2 4 5 1 0 0 2 5]


In [30]:
len(poscorpus)

100676

In [28]:
print(poscorpus[0:100])

['NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NNP', 'CD', '.', 'NNP', 'NNP', 'VBZ', 'NN', 'IN', 'NNP', 'NNP', ',', 'DT', 'NNP', 'VBG', 'NN', '.', 'NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', 'CC', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'VBD', 'VBN', '-NONE-', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'JJ', 'NN', '.', 'DT', 'NN', 'IN', 'NN', 'RB', 'VBN', '-NONE-', '-NONE-', 'TO', 'VB', 'NNP', 'NN', 'NNS', 'VBZ', 'VBN', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'DT', 'NN', 'IN', 'NNS', 'VBN', '-NONE-', 'TO', 'PRP', 'RBR', 'IN', 'CD', 'NNS', 'IN', ',', 'NNS', 'VBD', '-NONE-', '-NONE-', '.', 'DT']


In [21]:
pltsne(resp)

In [31]:
rnn=GRU_NLP(lsize,30,lsize)
rnnt=run_training(labels, lsize, rnn, 2000, learning_rate=1e-2, batch=20, window=30)

Perlexity:  200 5.468947524679297
Perlexity:  400 4.979690101682444
Perlexity:  600 5.090505573478588
Perlexity:  800 5.016363906903211
Perlexity:  1000 5.399086139434326
Perlexity:  1200 4.848065038602501
Perlexity:  1400 5.205955615176405
Perlexity:  1600 4.996428723954968
Perlexity:  1800 4.865840710532968
Time used in training: 51.12909483909607


In [38]:
res=concept_bind(poscorpus,labels)

NN [ 635. 1393. 1572.  653. 1229.  557. 6852.  275.]
IN [1441. 3468. 2902.  479.  192.  993.  150.  232.]
NNP [ 474.   88.  148. 3683. 1648. 1601. 1447.  321.]
DT [2093.   85.  326.   24. 3392. 1842.   29.  374.]
-NONE- [3050.  604. 1589.   59.  380.  771.   38.  101.]
NNS [ 396.  839. 1234.  358.  727.  371. 1930.  192.]
JJ [ 977.  182.  337.  114. 1044.  459. 2419.  302.]
, [ 152. 2048.  784. 1587.   25.    8.  108.  174.]
. [ 168. 1832. 1256.  526.   30.    3.   32.   27.]
CD [ 388.   49.  645.  197. 1490.  170.  513.   94.]
VBD [  14. 1681.   54.  665.    9.  329.   25.  266.]
RB [982. 414. 206.  80. 121. 514.  84. 421.]
VB [ 112.   44.   34.    8.  165.   19.    0. 2172.]
CC [ 83. 705. 354. 391.  11. 664.  48.   9.]
TO [ 502.  391. 1115.   47.   14.   24.    8.   78.]
VBN [1367.  214.   61.   11.   52.  129.  148.  152.]
VBZ [   5. 1089.   31.  375.    5.  258.   56.  306.]
PRP [424.  58.  91.   5. 327. 692.   6. 113.]
VBG [398. 169.  78.   8.  36. 118. 127. 526.]
VBP [2.000e+00 1

In [40]:
a=np.array([1,2,3])
a.argmax()

2

In [41]:
conceptb=dict([])
for k,v in res.items():
    ind=v.argmax()
    conceptb[k]=ind
print(conceptb)

{'NN': 6, 'IN': 1, 'NNP': 3, 'DT': 4, '-NONE-': 0, 'NNS': 6, 'JJ': 6, ',': 1, '.': 1, 'CD': 4, 'VBD': 1, 'RB': 0, 'VB': 7, 'CC': 1, 'TO': 2, 'VBN': 0, 'VBZ': 1, 'PRP': 5, 'VBG': 7, 'VBP': 1, 'MD': 1, 'POS': 3, 'PRP$': 4, '$': 4, '``': 5, "''": 5, ':': 1, 'WDT': 5, 'JJR': 0, 'NNPS': 3, 'WP': 5, 'RP': 0, 'JJS': 6, 'WRB': 5, 'RBR': 0, '-RRB-': 3, '-LRB-': 2, 'EX': 5, 'RBS': 6, 'PDT': 0, '#': 4, 'WP$': 5, 'LS': 5, 'FW': 2, 'UH': 0, 'SYM': 3}


In [44]:
concluster=[[] for _ in range(8)]
for k,v in conceptb.items():
    concluster[v].append(k)
print(concluster)

[['-NONE-', 'RB', 'VBN', 'JJR', 'RP', 'RBR', 'PDT', 'UH'], ['IN', ',', '.', 'VBD', 'CC', 'VBZ', 'VBP', 'MD', ':'], ['TO', '-LRB-', 'FW'], ['NNP', 'POS', 'NNPS', '-RRB-', 'SYM'], ['DT', 'CD', 'PRP$', '$', '#'], ['PRP', '``', "''", 'WDT', 'WP', 'WRB', 'EX', 'WP$', 'LS'], ['NN', 'NNS', 'JJ', 'JJS', 'RBS'], ['VB', 'VBG']]


In [45]:
datasetcon=[]
for item in poscorpus:
    datasetcon.append(conceptb[item])
print(datasetcon[0:100])

[3, 3, 1, 4, 6, 6, 1, 1, 7, 4, 6, 1, 4, 6, 6, 3, 4, 1, 3, 3, 1, 6, 1, 3, 3, 1, 4, 3, 7, 6, 1, 3, 3, 1, 4, 6, 6, 1, 6, 6, 1, 3, 3, 3, 3, 1, 1, 0, 0, 4, 6, 6, 1, 4, 6, 6, 6, 1, 4, 6, 1, 6, 0, 0, 0, 0, 2, 7, 3, 6, 6, 1, 0, 4, 6, 6, 1, 6, 6, 1, 4, 6, 1, 6, 0, 0, 2, 5, 0, 1, 4, 6, 1, 1, 6, 1, 0, 0, 1, 4]


In [46]:
rnn=GRU_NLP(8,30,8)
rnnt=run_training(datasetcon, 8, rnn, 2000, learning_rate=1e-2, batch=20, window=30)

Perlexity:  200 4.54388341592718
Perlexity:  400 4.277102523085104
Perlexity:  600 4.408851418931514
Perlexity:  800 4.519881578560373
Perlexity:  1000 4.1863090006197154
Perlexity:  1200 4.056519814694535
Perlexity:  1400 4.178965485999017
Perlexity:  1600 4.0414405496166
Perlexity:  1800 4.061318746380808
Time used in training: 37.42580509185791


In [36]:
def concept_bind(poscorpus,labels):
    res=dict([])
    for ii in range(lsize):
        res[nlp.id_to_word[ii]]=np.zeros(8)
    for iit in range(len(poscorpus)):
        res[poscorpus[iit]][labels[iit]]=res[poscorpus[iit]][labels[iit]]+1
    for k,v in res.items():
        print(k,v)
    return res

In [3]:
nlp=NLPutil()
nlp.get_data("ptb")
poscorpus=[]
for sent in nlp.tagged_sents:
    for tup in sent:
        poscorpus.append(tup[1])
print(len(poscorpus))
t2id,id2t=nlp.build_vocab(poscorpus)
t_w2v=nlp.build_w2v(mode="onehot")
t_mat=nlp.build_textmat(poscorpus)
nlp.plot_txtmat(t_mat.T,start=0,length=100,text=poscorpus)
dataset=[]
for wrd in nlp.corpus:
    dataset.append(nlp.word_to_id[wrd])
lsize=len(nlp.id_to_word)

Initializing NLPUtil
Getting corpus data...
Length of corpus: 100676
Vocabulary of corpus: 11387
100676
Building vocabulary...
{0: 'NN', 1: 'IN', 2: 'NNP', 3: 'DT', 4: '-NONE-', 5: 'NNS', 6: 'JJ', 7: ',', 8: '.', 9: 'CD', 10: 'VBD', 11: 'RB', 12: 'VB', 13: 'CC', 14: 'TO', 15: 'VBN', 16: 'VBZ', 17: 'PRP', 18: 'VBG', 19: 'VBP', 20: 'MD', 21: 'POS', 22: 'PRP$', 23: '$', 24: '``', 25: "''", 26: ':', 27: 'WDT', 28: 'JJR', 29: 'NNPS', 30: 'WP', 31: 'RP', 32: 'JJS', 33: 'WRB', 34: 'RBR', 35: '-RRB-', 36: '-LRB-', 37: 'EX', 38: 'RBS', 39: 'PDT', 40: '#', 41: 'WP$', 42: 'LS', 43: 'FW', 44: 'UH', 45: 'SYM'}
Building word to vector lookup table...
Building sequecing vector of text...
